# Classifying MNIST

In [ ]:
import os
import sys
module_path = os.path.abspath(os.path.join('../../fmriDEEP'))
if module_path not in sys.path:
    sys.path.append(module_path)

<p align="justify">We can now import the necessary packages and tools to train a simple linear classifier with PyTorch and my custom code.</p>

In [ ]:
# other packages
import torch
import numpy as np
from torchinfo import summary
from sklearn.model_selection import StratifiedKFold
from torchvision.transforms import ToTensor
from torch.utils.data import DataLoader, Subset, Dataset
from torchvision.datasets import MNIST
import matplotlib.pyplot as plt

# custom packages
from _core.utils.train_fns import standard_train # this is a predefined training function we will use
from _core.utils.tools import compute_accuracy   # a custom function to compute the accuracy scores
from _core.networks.LinearNets import SimpleLinearModel
from _core.networks.ConvNets import Simple2dCnnClassifier

# this variable contains information whether a GPU can be used for training. If not, we automatically use the CPU.
DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

<p align="justify">To ensure reproducibility we set the random seed for all sorts of randomizer tools.</p>

In [ ]:
# set the random seed for reproducibility
def set_random_seed(seed):
    import random 
    
    torch.manual_seed(seed)
    random.seed(seed)
    np.random.seed(seed)
    g = torch.Generator() # can be used in pytorch dataloaders for reproducible sample selection when shuffle=True
    g.manual_seed(seed)
    
    return g

g = set_random_seed(42)

## Download and inspect the MNIST data

The easiest benchmark test for any neural network is to test its classification performance on the MNIST dataset.
This dataset contains thousands of hand-written digit exemplars. 
As it is quite easy to use, we will employ it here as well.

In [ ]:
mnist_train = MNIST('./data/', train=True, download=True if not os.path.exists('./data/MNIST') else False, transform=ToTensor())
mnist_test = MNIST('./data/', train=False, download=False, transform=ToTensor())

Let us look at some examples from the dataset:

In [ ]:
fig, axes = plt.subplots(1,4, figsize=(12,6), sharex=True, sharey=True)
for i, ax in enumerate(axes):
    ax.imshow(mnist_train.data[i], cmap='gray')

<p align="justify">What you also always should do is look at your data or something that data scientists would call 'Exploratory Data Analysis' (EDA). EDA usually can be quite extensive and they are- for now -too much for the scope of this book. 

We can now turn our attention to creating the network and setting up the training procedure.</p>

## Creating a simple linear neural network

<p align="justify">As demonstrated in the previous chapter, it is fairly easy to create a default linear network with the toolbox.</p>

In [ ]:
# For the simplest version of a linear neural network this is all you have to do:
model = SimpleLinearModel(784, 10)

# we print the model summary here again.
print(summary(model, (1, 784)))

<p align="justify">Now that we know a little bit about the data and we defined our network it is now time to train the network.</p>

## Training the model to classify hand-written digits

As the goal of the toolbox is to make it as easy as possible for you, the user, it comes with a predefined ```standard_train``` function. The code below (click the button to expand) shows you what this function looks like and, again, for ease of use, this function is being used by default when you create any model of the toolbox (e.g., ```SimpleLinearModel```, ```Simple2dCnnClassifier```, etc).

In [ ]:
# the standard_train function
# it requires some additional packages that we import here
# in case you simply want to use it you do not have to worry about
# implementing these packages, the toolbox does this automatically for you.
# We just show it here for completeness
from torch.nn import CrossEntropyLoss
from torch.utils.data import DataLoader
from torch.optim import Adam
from typing import Tuple


def standard_train(
        model,
        train_data: DataLoader,
        loss_fn=CrossEntropyLoss(),
        optimizer=Adam, lr: float = .00001,
        device: torch.device = torch.device("cpu"),
        train=True,
        **optimizer_kwargs
) -> Tuple[np.ndarray, np.ndarray]:
    r"""
    A simple function to train a supplied neural network for multiclass problems.

    Args:
        model: only necessary if this function is not supplied to the model constructor. Otherwise model.fit()
            calls the function with model=self
        train_data (DataLoader): dataloader used for training or validation/test if function is called in model.eval()
            context
        loss_fn: loss function to use (default: CrossEntropyLoss)
        optimizer: optimizer to use to adjust weights in backward pass (default: Adam)
        lr (float): the learning rate for the optimizer
        device (torch.device): do computations on device, e.g., cpu or gpu (default: cpu)
        train (bool): set the network into training mode (True|default) or evaluation (False)
        **optimizer_kwargs: additional arguments for the supplied optimizer

    Returns:
        Tuple[epoch_loss, stats]: the stats variable contains multiple values. The first 0:n_classes columns contains
        the classification probability for a given input (row). The second to last column (i.e., stats[:,-2]) contains
        the predicted label. The last column (i.e. stats[:, -1] contains the real label.

    """

    # set the model into training mode if it is not and train=True
    if train:
        if not model.training:
            model.train()
    else:
        model.eval()

    # here is something new: the optimizer
    # The optimizer determines the algorithm with which the weights of the layers
    # are adjusted. Here we use the 'Adam' algorithm by default.
    optimizer = optimizer(model.parameters(), lr=lr, **optimizer_kwargs)

    epoch_loss = 0

    # the batch loop. Within this loop we iterate over all samples stored in the
    # train_data variable.
    # the variable 'batch' represents the current iteration [integer value]
    # the variable 'inputs' is the actual input data in the shape of batch_size-by-inputshape
    # the variable 'labels' contains the respective class label
    for batch, (inputs, labels) in enumerate(train_data):

        # in this function we transfer the data and the label tensors to the chosen device
        # NOTE: I RECOMMEND TRANSFERING THE DATA AND LABELS TO THE DEVICE WHEN YOU LOAD THEM
        # MORE ON THIS IN A DIFFERENT STEP THOUGH [see DataSets and DataLoaders)
        inputs, labels = inputs.to(device), labels.to(device)

        # reset the gradients
        # a crucial step in training the networks. Otherwise the gradients accumulate after
        # each batch iteration and weird stuff will happen.
        if model.training:
            for p in model.parameters():
                p.grad = None

        outputs = model(inputs.float())  # forward pass through model

        loss = loss_fn(outputs.squeeze(), labels.squeeze())  # calculate loss

        # in this statement we check if the network is currently in training mode,
        # which means, that every layer in the network has the required_grad flag set
        # to True. This in turn means that the backpropagation algorithm is executed.
        # If the model, however, is not in training mode we do not want to exectue
        # the backward pass and we also do not want to store the so-called pytorch graph.
        if model.training:
            loss.backward()  # do a backward pass
            optimizer.step()  # update parameters

        # get the probabilities of the predictions
        prediction_probs = model.SM(outputs.data).cpu().numpy()

        # get the label number of the output
        _, predicted_labels = torch.max(outputs.data, 1)

        epoch_loss += loss.item()  # sum up the loss over all batches

        # just a helper variable
        inter = np.hstack(
            [prediction_probs.squeeze(),
             labels.squeeze().cpu().numpy()[:, None],
             predicted_labels.squeeze().cpu().numpy()[:, None]]
        )
        stats = np.vstack([stats, inter]) if 'stats' in vars() else inter  # noqa

    return epoch_loss / len(train_data), stats  # noqa

Okay. Now that we know a little bit about how our model is trained, we need to do some preparation work first.
That means we will download the MNIST dataset and then setup a training loop.

Let's do that!

### Define the training procedure

We should first make sure that our network is also on our detected device. In case you do not send your model or your data to the same device you will run into errors. 

Next, we should determine for how long, that is how many *epochs*, we want to train our model.

In [ ]:
# first, we should make sure the network is on the correct device:
model.to(DEVICE);

# define the number of epochs (that is training iterations) you want your model to go through
# let's say we want this to be 10 epochs
n_epochs = 10

```{warning}
Make sure you only send your model to a device *once*. If you have multiple lines of this model.to(DEVICE) it can lead to unwanted results. I speak from experience
```

Now we setup the dataloaders for the training function. In case you have not gone through the tutorial on {ref}`dataloader <dataloader_chapter>` I suggest you do this at some point to undestand what they are good for and how to define them.

The PyTorch community has a really good introduction on their [offcial website](https://pytorch.org/tutorials/beginner/basics/data_tutorial.html).

In [ ]:
# create the dataloaders
dl_train = DataLoader(mnist_train, batch_size=256, shuffle=True, generator=g)
dl_test = DataLoader(mnist_test, batch_size=256, shuffle=True, generator=g)

In [ ]:
# start training the network

# set some variables here, such that we can create pretty plots
train_loss = np.zeros(n_epochs)
test_loss = np.zeros_like(train_loss)
train_acc = np.zeros_like(train_loss)
test_acc = np.zeros_like(train_loss)

# loop for the above set number of epochs
for epoch in range(0, n_epochs):

    # THIS IS WHERE THE MAGIC HAPPENS
    # calling the model.fit() function will execute the 'standard_train' function as defined above.
    train_loss[epoch], train_stats = model.fit(dl_train, lr=.001, device=DEVICE)
    train_acc[epoch] = compute_accuracy(train_stats[:, -1], train_stats[:, -2])

    # for validating or testing set the network into evaluation mode such that layers like dropout are not active
    with torch.no_grad():
        test_loss[epoch], test_stats = model.fit(dl_test, device=DEVICE, train=False)
        test_acc[epoch] = compute_accuracy(test_stats[:, -1], test_stats[:, -2])

    print('epoch=%03d, train_loss=%1.3f, train_acc=%1.3f, test_loss=%1.3f, test_acc=%1.3f' % 
         (epoch, train_loss[epoch], train_acc[epoch], test_loss[epoch], test_acc[epoch]))

# let's save the model
model.save('my_lin_mnist', save_full=True)

That was not so bad was it? Quite simple in fact, I hope. <br>
Please keep in mind that we try to incorporate the epoch loop into the model.fit() function as well. <br>For now, we unfortunately encounter some memory leaks that we first need to solve.

### Check model performance

Let's look if our network really learned something. For that we will plot some loss and accuracy scores as a function of the epoch as well as a confusion matrix for the test data. 

In [ ]:
from _core.utils.plots import confusion_matrix

fig, axes = plt.subplots(1,3,figsize=(16,6))
axes[0].plot(range(1,n_epochs+1), train_loss, range(1,n_epochs+1), test_loss, linewidth=3, marker="o", markersize=10, markeredgecolor="white")
axes[0].set_xlabel('Epoch'); axes[0].set_ylabel('CrossEntropyLoss'), axes[0].spines['top'].set_visible(False), axes[0].spines['right'].set_visible(False);
axes[0].legend(("train", "test"));

axes[1].plot(range(1,n_epochs+1), train_acc, range(1,n_epochs+1), test_acc, linewidth=3, marker="o", markersize=10, markeredgecolor="white")
axes[1].axhline(.1, color='black', linestyle='--')
axes[1].set_xlabel('Epoch'); axes[1].set_ylabel('Accuracy'), axes[1].spines['top'].set_visible(False), axes[1].spines['right'].set_visible(False);
axes[1].legend(("train", "test", "chance"));

lin_conf_mat = confusion_matrix(test_stats[:,-2], test_stats[:,-1], range(0,10), ax=axes[2])

This looks great! The line plots clearly indicate a downard trend in the loss curve (left) and an upward trend in the accuracy curve, respectively. Further, we can see that the test loss is lower than in the train. This is somewhat interesting as usually the training set performs better than the test or validation data. It is important to note that the loss and accuracy scores are averages for a given epoch. The training set is about 6 times larger than the test set in the case of MNIST. This is only one explanation of why we get curves like these.

Now, let me show you how easy it is to switch to another type of neural network: a convolutional neural network (CNN) model

## Using a CCN

Changing to a CNN with the toolbox is quite simple. Instead of the ```SimpleLinearModel``` we now use the ```Simple2dCnnClassifier```.

As the name suggest we are now working with data that is 2-dimensional. Luckily for us the MNIST images are just that. We also know that the MNIST images have 28-by-28 pixels. Thus our input_dims for the model are now [28, 28]. The number of classes remains at 10.

We also again can use the ```summary``` function to better understand how the model is configured. 

```{note}
Images usually have color channels, thus the input of our images needs to be (batch, channels, pixeldim1, pixeldim2)
```

In [ ]:
# we actually do exactly the same as for the linear model. 
model = Simple2dCnnClassifier((28, 28), 10)
print(summary(model, (1, 1, 28, 28)))

Alrighty. Make sure you send the model to the device. We can keep the number of epochs and the dataloader the same as for the ```SimpleLinearModel```.

In [ ]:
# first, we should make sure the network is on the correct device:
model.to(DEVICE);

Ok! Time to train your first 2-D CNN!
    
Notice that the code below essentially is the same as it is for the linear model. The only thing that changed is some variable names such that we can compare them. But isn't that great? If we find a nice way we turn the below code into a function and simply supply a model to it and it runs. No matter the network.

```{note}
I also changed the learning rate from .001 to .01. Interesting effects can happen if one decreases or increases the learning rate. It is quite common however, that for deeper CNNs larger learning rates work better. 
```

Finding the best, or at least a good combination of parameters is what we cover in the {ref}`next chapter <using_sweeps>`.

In [ ]:
# start training the network

# set some variables here, such that we can create pretty plots
cnn_train_loss = np.zeros(n_epochs)
cnn_test_loss = np.zeros_like(train_loss)
cnn_train_acc = np.zeros_like(train_loss)
cnn_test_acc = np.zeros_like(train_loss)

# loop for the above set number of epochs
for epoch in range(0, n_epochs):

    # THIS IS WHERE THE MAGIC HAPPENS
    # calling the model.fit() function will execute the 'standard_train' function as defined above.
    cnn_train_loss[epoch], cnn_train_stats = model.fit(dl_train, lr=.01, device=DEVICE)
    cnn_train_acc[epoch] = compute_accuracy(cnn_train_stats[:, -1], cnn_train_stats[:, -2])

    # for validating or testing set the network into evaluation mode such that layers like dropout are not active
    with torch.no_grad():
        cnn_test_loss[epoch], cnn_test_stats = model.fit(dl_test, device=DEVICE, train=False)
        cnn_test_acc[epoch] = compute_accuracy(cnn_test_stats[:, -1], cnn_test_stats[:, -2])

    print('epoch=%03d, train_loss=%1.3f, train_acc=%1.3f, test_loss=%1.3f, test_acc=%1.3f' % 
         (epoch, cnn_train_loss[epoch], cnn_train_acc[epoch], cnn_test_loss[epoch], cnn_test_acc[epoch]))

# Let's save the model
model.save('my_cnn_mnist', save_full=True)

Very nice. Let us see how the CNN performes compared to the linear network!

In [ ]:
from _core.utils.plots import confusion_matrix

fig, axes = plt.subplots(1,3,figsize=(16,6))
axes[0].plot(range(1,n_epochs+1), train_loss, 
             range(1,n_epochs+1), test_loss, linewidth=3, marker="o", markersize=10, markeredgecolor="white")
axes[0].plot(range(1,n_epochs+1), cnn_train_loss,
             range(1,n_epochs+1), cnn_test_loss, linewidth=3, linestyle=":", marker="o", markersize=10, markeredgecolor="white")
axes[0].set_xlabel('Epoch'); axes[0].set_ylabel('CrossEntropyLoss'), axes[0].spines['top'].set_visible(False), axes[0].spines['right'].set_visible(False);
axes[0].legend(("lintrain", "lintest", "cnntrain", "cnntest"));

axes[1].plot(range(1,n_epochs+1), train_acc, 
             range(1,n_epochs+1), test_acc, linewidth=3, marker="o", markersize=10, markeredgecolor="white")
axes[1].plot(range(1,n_epochs+1), cnn_train_acc,
             range(1,n_epochs+1), cnn_test_acc, linewidth=3, linestyle=":", marker="o", markersize=10, markeredgecolor="white")
axes[1].axhline(.1, color='black', linestyle='--')
axes[1].set_xlabel('Epoch'); axes[1].set_ylabel('Accuracy'), axes[1].spines['top'].set_visible(False), axes[1].spines['right'].set_visible(False);
axes[1].legend(("lintrain", "lintest", "cnntrain", "cnntest", "chance"));

cnn_conf_mat = confusion_matrix(cnn_test_stats[:,-2], cnn_test_stats[:,-1], range(0,10), ax=axes[2])

<p align="justify">It seems that the default CNN outperforms the default linear network. Pretty cool. However, we can also see that at about epoch 9 the train loss for the CNN seems to increase again. This might already indicate some overfitting issues. Something to bear in mind! 

The graph below shows the difference of the CNN confusion matrix minus the LN confusion matrix. We can immediately see that the CNN outperforms the LN because the diagonal clearly has positive values in the majority of the classes. Remember: the diagonal means that the network correctly predicts the real class.</p>

In [ ]:
fig, ax = plt.subplots(1,1,figsize=(12,6))
a = ax.imshow(cnn_conf_mat-lin_conf_mat, cmap='seismic');
plt.colorbar(a);
ax.set_title('Diff CNN_conf - Lin_conf');

## Exercise

* Try using the default (or your own configuration) of the ```SimpleLinearModel``` and the ```Simple2dCnnClassifier``` on the Fashion-MNIST dataset. See which performs better!